In [1]:
import torch
import resnetMod
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
from MyConvLSTMCell import *


class attentionModel(nn.Module):
    def __init__(self, num_classes=61, mem_size=512):
        super(attentionModel, self).__init__()
        self.num_classes = num_classes
        self.resNet = resnetMod.resnet34(True, True)
        self.mem_size = mem_size
        self.weight_softmax = self.resNet.fc.weight
        self.lstm_cell = MyConvLSTMCell(512, mem_size)
        self.avgpool = nn.AvgPool2d(7)
        self.dropout = nn.Dropout(0.7)
        self.fc = nn.Linear(mem_size, self.num_classes)
        self.classifier = nn.Sequential(self.dropout, self.fc)

    def forward(self, inputVariable, img, size_upsample):
        state = (Variable(torch.zeros((inputVariable.size(1), self.mem_size, 7, 7)).cuda()),
                 Variable(torch.zeros((inputVariable.size(1), self.mem_size, 7, 7)).cuda()))
        for t in range(inputVariable.size(0)):

          logit, feature_conv, feature_convNBN = self.resNet(inputVariable[t])
          bz, nc, h, w = feature_conv.size()
          feature_conv = feature_conv.view(bz, nc, h*w)
          h_x = F.softmax(logit, dim=1).data
          probs, idx = h_x.sort(1, True)
          cam_img = torch.bmm(self.weight_softmax[idx[:, 0]].unsqueeze(1), feature_conv).squeeze(1)
          cam_img = F.softmax(cam_img, 1).data 
          cam_img = cam_img.cpu().numpy()
          cam_img = cam_img.reshape(h, w)
          cam_img = cam_img - np.min(cam_img)
          cam_img = cam_img / np.max(cam_img)
          cam_img = np.uint8(255 * cam_img)
          output_cam = cv2.resize(cam_img, size_upsample)
          img = cv2.cvtColor(np.uint8(img), cv2.COLOR_RGB2BGR)
          heatmap = cv2.applyColorMap(output_cam, cv2.COLORMAP_JET)
          result = heatmap * 0.3 + img * 0.5

        '''
        for t in range(inputVariable.size(0)):
            logit, feature_conv, feature_convNBN = self.resNet(inputVariable[t])
            bz, nc, h, w = feature_conv.size()
            feature_conv1 = feature_conv.view(bz, nc, h*w)
            probs, idxs = logit.sort(1, True)
            class_idx = idxs[:, 0]
            cam = torch.bmm(self.weight_softmax[class_idx].unsqueeze(1), feature_conv1)
            cam_img = F.softmax(cam, 1).data
            cam_img = cam_img.cpu().numpy()
            cam_img = cam_img.reshape(h, w)
            cam_img = cam_img - np.min(cam_img)
            cam_img = cam_img / np.max(cam_img)
            cam_img = np.uint8(255 * cam_img)
            output_cam = cv2.resize(cam_img, size_upsample)
            img = cv2.cvtColor(np.uint8(img), cv2.COLOR_RGB2BGR)
            heatmap = cv2.applyColorMap(output_cam, cv2.COLORMAP_JET)
            result = heatmap * 0.3 + img * 0.5
            print(result)
        '''

ModuleNotFoundError: ignored